In [1]:
import geopandas as gpd
import pandas as pd
from folium.plugins import HeatMapWithTime
import folium


In [2]:
spain_provinces_gdf = gpd.read_file("../data/heatmap/spain-provinces.geojson")
precipitation_df = pd.read_excel("../data/heatmap/tabla_clima_heatmap.xlsx")

# Buscamos en el archivo geojson la relación entre los nombre de provincias
province_mapping = {
    "a_coruna": "A Coruña",
    "almeria": "Almería",
    "cadiz": "Cádiz",
    "huelva": "Huelva",
    "girona": "Girona"
}
precipitation_df['provincia_mapped'] = precipitation_df['provincia'].str.lower().map(province_mapping)

# Filtramos las filas que no sean de las 5 provincias de la muestra
precipitation_df_filtered = precipitation_df[precipitation_df['provincia_mapped'].notna()]

# Merge para combinar los datos de precipitación con los datos geográficos por el nombre de provincia
merged_gdf = spain_provinces_gdf.merge(precipitation_df_filtered, left_on="name", right_on="provincia_mapped")

In [3]:
# Mapa base de España
m = folium.Map(location=[40.4168, -3.7038], zoom_start=6)

# Extrae los datos de heatmap con el tiempo
locations = []
for date, group in merged_gdf.groupby('fecha'):
    locations_per_date = [[row.geometry.centroid.y, row.geometry.centroid.x, row.p_mes] for _, row in group.iterrows()]
    locations.append(locations_per_date)


hm = HeatMapWithTime(locations, radius=20, gradient={0.4: 'orange', 0.6: 'lime', 1: 'blue'}, min_opacity=0.5, max_opacity=1, use_local_extrema=True)
hm.add_to(m)

m


In [4]:
m.save("../data/heatmap/heatmap.html")

In [5]:
m.save("../data/heatmap/heatmap.gif")